<a href="https://colab.research.google.com/github/guptapallavi30/ScalableML_1/blob/main/TextHypothesisClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install kaggle

# Upload your Kaggle API key file to "/content/kaggle/kaggle.json"
# Replace "/content/kaggle/kaggle.json" with the path where you placed your Kaggle API key file in Colab.
!mkdir -p ~/.kaggle
!cp "../content/kaggle/kaggle.json" ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
import pandas as pd

# Replace "narendrageek/my-dear-watson-translated-text" with the dataset name you want to use.
!kaggle datasets download -d narendrageek/my-dear-watson-translated-text -p /content

# Unzip the downloaded dataset (if necessary).
import zipfile
with zipfile.ZipFile("/content/my-dear-watson-translated-text.zip", "r") as zip_ref:
    zip_ref.extractall("/content")

# Read the dataset into a pandas DataFrame.
df = pd.read_csv("/content/translated_train_dataset.csv")

# Now, you can work with the DataFrame df without downloading the dataset to your local machine.


my-dear-watson-translated-text.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
df.head()

,id,premise,hypothesis,lang_abv,language,label,premise_translated,hypothesis_translated
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0,and these comments were considered in formulat...,The rules developed in the interim were put to...
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0,Little things like these make a huge differenc...,I was trying to accomplish something.
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0,you know they can't really defend themselves l...,They can't defend themselves because of their ...
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1,In role playing as well The opportunity to exp...,Children can see how different ethnic groups are.
